In [6]:
#Load Data
import numpy as np
import pandas as pd

import statsmodels
import statsmodels.api as sm
from statsmodels.tsa.stattools import coint
from IPython.display import display

import matplotlib.pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

import plotly.graph_objects as go
import yfinance as yf
from Quantapp.Computation import Computation
from Quantapp.Algorithm   import Algorithm
comp = Computation()
algorithm = Algorithm()
plt.rcParams["figure.figsize"] = (20, 7)

time_frame_week = 7
time_frame_short = 21
time_frame_mid   = 50
time_frame_long = 200


period     = '5y'

risk_free_rate = 0.02 / 252  # Annualized risk-free rate divided by trading days

def calculate_sortino_ratio(returns):
    excess_returns = returns - risk_free_rate
    downside_deviation = excess_returns[excess_returns < 0].std()
    sortino_ratio = excess_returns.mean() / downside_deviation if downside_deviation != 0 else np.nan
    return sortino_ratio

def calculate_risk_adjusted_returns(df, time_frame):
    daily_returns = df.pct_change()
    rolling_sortino_ratio = daily_returns.rolling(window=time_frame).apply(calculate_sortino_ratio)
    return rolling_sortino_ratio

def generate_series(arr):
    data = {}
    for ticker in arr:
        yf_ticker = ticker.replace('.', '-')
        data[ticker] = yf.Ticker(yf_ticker).history(period=period)['Close']
    df = pd.DataFrame(data)
    return df


def plot_returns(returns, time_frame):
    threshold = 0
    fig = px.line(returns , x=returns.index, y=returns.columns)
    fig.add_hline(y=threshold, line_dash="dash", line_color="red", annotation_text=f"y={threshold}")
    fig.add_hline(y=threshold, line_dash="dash", line_color="red", annotation_text=f"y={threshold}")
    fig.show()

    
def create_spreads(asset_series, benchmark_series, time_frame, mode='standard'):
    
    if mode == 'standard':
        asset_returns = asset_series.pct_change(time_frame)
        benchmark_returns= benchmark_series.pct_change(time_frame)
    elif mode == 'sortino':
        asset_returns = calculate_risk_adjusted_returns(asset_series, time_frame)
        benchmark_returns= calculate_risk_adjusted_returns(benchmark_series, time_frame)

    benchmark_minus_asset = asset_returns.apply(lambda x: benchmark_returns - x)
    benchmark_minus_asset.columns = ["Benchmark" + "_minus_" + col for col in benchmark_minus_asset.columns]
    return benchmark_minus_asset


def create_spread_plot(asset_spreads):
    spread_threshold = 0
    spread           = asset_spreads
    mean             = spread[spread>=0].mean()
    std_dev = spread[spread >= 0].std()
    #spread = asset_spreads[:200]


    fig = px.line(spread)
    fig.update_layout(title=asset_spreads.name)
    fig.add_hline(y=spread_threshold, line_dash="dash", line_color="red", annotation_text=f"y={spread_threshold}")
    fig.add_hline(y=mean , line_color="red", annotation_text="mean")
    fig.add_hline(y=mean + std_dev, line_dash="dash", line_color="blue", 
                  annotation_text="mean + 1 std dev", annotation_position="bottom right")
    fig.add_hline(y=mean - std_dev, line_dash="dash", line_color="blue", 
                  annotation_text="mean - 1 std dev", annotation_position="bottom right")
    fig.add_hline(y=mean + 2*std_dev, line_dash="dot", line_color="green", 
                  annotation_text="mean + 2 std dev", annotation_position="bottom right")
    fig.add_hline(y=mean - 2*std_dev, line_dash="dot", line_color="green", 
                  annotation_text="mean - 2 std dev", annotation_position="bottom right")
    fig.add_shape(type="rect",
                  xref="paper", yref="y",
                  x0=0, y0=mean, x1=1, y1=spread.max(),
                  fillcolor="green", opacity=0.2, line_width=0)
    fig.update_layout(height=800)
    return fig

def create_side_by_side_subplots(fig1, fig2):
    fig = make_subplots(rows=1, cols=2, subplot_titles=(fig1.layout.title.text, fig2.layout.title.text))
    
    for trace in fig1.data:
        fig.add_trace(trace,row=1,col=1)

    for trace in fig2.data:
        fig.add_trace(trace,row=1,col=2)
    
    return fig

def plot_multiple_spreads(assets):
    for column in assets:
       asset_spreads = assets[column]
       create_spread_plot(asset_spreads).show()

def plot_risk_adjusted_returns(series,time_frame):
    series_adjusted_returns = calculate_risk_adjusted_returns(series,time_frame)
    negative_returns = series_adjusted_returns[series_adjusted_returns<0]
    mean = negative_returns.mean()
    standard_deviation = negative_returns.std()
    standard_deviation_level_three_fourths = mean - .5 * standard_deviation
    standard_deviation_level_single        = mean - standard_deviation

    fig = px.line(series_adjusted_returns)

    fig.add_hline(y=0, line_dash="dash", line_color="black", 
                annotation_text="Zero Line", annotation_position="bottom right")
    fig.add_hline(y=mean, line_dash="dot", line_color="blue", 
                annotation_text=f"Mean of negative returns: {mean:.2f}", annotation_position="top right")
    fig.add_hline(y=standard_deviation_level_three_fourths , line_dash="dashdot", line_color="red", 
                annotation_text=f".75 Std Dev: {standard_deviation_level_three_fourths :.2f}", annotation_position="top right")
    fig.add_hline(y=standard_deviation_level_single, line_dash="dashdot", line_color="red", 
                annotation_text=f"1 Std Dev: {standard_deviation_level_single:.2f}", annotation_position="top right")

    fig.add_shape(
        type="rect",
        x0=series_adjusted_returns.index.min(),
        x1=series_adjusted_returns.index.max(),
        y0=standard_deviation_level_three_fourths,
        y1=standard_deviation_level_single,
        fillcolor="green",
        opacity=0.2,
        line_width=0,
    )

    return fig

SPY = 'SPY'
IWM = 'IWM'
QQQ = 'QQQ'
DIA = 'DIA'
TLT = 'TLT'
AGG = 'AGG'
GLD = 'GLD'
XLK = 'XLK'
XLF = 'XLF'
benchmark = SPY

INDICES          = ['SPY','QQQ','DIA','IWM']
SECTORS          = ['SPY','XLF','XLK','XLV','XLC','XLI','XLU','XLB','VNQ','XLP','XLY','XBI','XLE']
INDUSTRIES       = ['SPY', 'SMH', 'KRE','KIE', 'KBE']
TOP_SPY_HOLDINGS = ['SPY','MSFT', 'AAPL', 'NVDA', 'AMZN', 'META', 'GOOG', 'BRK.B', 'AVGO','JPM','XOM','TSLA','UNH','V']
TOP_QQQ_HOLDINGS = ['QQQ','MSFT', 'AAPL', 'NVDA', 'AMZN', 'META', 'GOOG', 'BRK.B', 'AVGO', 'COST', 'TSLA', 'NFLX','PEP','AMD','ADBE','LIN',
                    'QCOM','CSCO','TMUS','INTU','AMAT','TXN','AMGN','CMCSA', 'MU']
TOP_DIA_HOLDINGS = ['DIA','UNH','GS','MSFT','CAT','HD', 'AMGN', 'V', 'CRM', 'MCD', 'AXP','TRV','HON','JPM','AMZN',
                    'AAPL','BA','IBM','PG','CVX','JNJ','MRK','DIS','MMM','NKE','KO','WMT','DOW','CSCO','VZ','INTC']
TOP_XLK_HOLDINGS = ['XLK','MSFT','AAPL','AVGO','NVDA','CRM','AMD','ADBE','QCOM','CSCO','ACN','ORCL','INTU','AMAT','TXN',
                    'IBM','NOW', 'MU', 'INTC', 'LRCX','ADI','KLAC','PANW']
TOP_XLF_HOLDINGS = ['XLF','BRK.B', 'JPM','V','MA','BAC','WFC','GS','SPGI','AXP','PGR','MS','C','SCHW','CB','MMC','FI','BX','ICE','CME','PYPL','USB',
                    'PNC','MCO','AON','AIG']
BONDS            = ['AGG','IEF','TLT', 'HYG','LQD']
PRECIOUS_METALS  = ['GLD','SLV','GDX','XME']
CRYPTO           = ['BITO','BLOK']
ENERGY           = ['USO','UNG','OIH','XOP','TAN','ICLN','URA','URNM','GUSH','KOLD']
CAPITALIZATIONS  = ['SPY', 'IJH' , 'IJR']
INNOVATION       = ['ARKG','ARKF','ARKK']
LONG_LEVERAGE    = ['TQQQ','SOXL','SPXL','TNA','BOIL','NUGT','ERX','DPST']
SHORT_LEVERAGE   = ['SQQQ','SPXS','UDOW','SSO','TECL','FAS','NVDA','TQQQ', 'VXX','UVXY','VIXY','UVIX','SVXY','SOXS','TZA','USD','TSLL','LABU','DPST','NUGT','CONL']
FOREIGN_MARKETS  = ['EWZ','EWJ','EWA','EWG','EWW','EEM','EFA','FEZ','INDA','EWU','EWG']






In [7]:
indices_series             = generate_series(INDICES)
sectors_series             = generate_series(SECTORS)
industries_series          = generate_series(INDUSTRIES)
top_spy_holdings_series    = generate_series(TOP_SPY_HOLDINGS)
top_qqq_holdings_series    = generate_series(TOP_QQQ_HOLDINGS)
top_dia_holdings_series    = generate_series(TOP_DIA_HOLDINGS)
top_xlk_holdings_series    = generate_series(TOP_XLK_HOLDINGS)
top_xlf_holdings_series    = generate_series(TOP_XLF_HOLDINGS) 
benchmark_series           = indices_series[benchmark]

all_series = pd.concat([
    indices_series,
    sectors_series,
    industries_series,
    top_spy_holdings_series,
    top_qqq_holdings_series,
    top_dia_holdings_series,
    top_xlk_holdings_series,
    top_xlf_holdings_series,
    benchmark_series
], axis=1)


SPY: No price data found, symbol may be delisted (period=5y)
QQQ: No price data found, symbol may be delisted (period=5y)
DIA: No price data found, symbol may be delisted (period=5y)
IWM: No price data found, symbol may be delisted (period=5y)
SPY: No price data found, symbol may be delisted (period=5y)
XLF: No price data found, symbol may be delisted (period=5y)
XLK: No price data found, symbol may be delisted (period=5y)
XLV: No price data found, symbol may be delisted (period=5y)
XLC: No price data found, symbol may be delisted (period=5y)
XLI: No price data found, symbol may be delisted (period=5y)
XLU: No price data found, symbol may be delisted (period=5y)
XLB: No price data found, symbol may be delisted (period=5y)
VNQ: No price data found, symbol may be delisted (period=5y)
XLP: No price data found, symbol may be delisted (period=5y)
XLY: No price data found, symbol may be delisted (period=5y)
XBI: No price data found, symbol may be delisted (period=5y)
XLE: No price data found

In [8]:

mode='standard'

benchmark_minus_indices_week          = create_spreads(indices_series, benchmark_series, time_frame=time_frame_week,mode=mode)
benchmark_minus_sectors_week          = create_spreads(sectors_series, benchmark_series, time_frame=time_frame_week,mode=mode)
benchmark_minus_industries_week       = create_spreads(industries_series, benchmark_series, time_frame=time_frame_week,mode=mode)
benchmark_minus_top_spy_holdings_week = create_spreads(top_spy_holdings_series, benchmark_series, time_frame=time_frame_week,mode=mode)
benchmark_minus_top_qqq_holdings_week = create_spreads(top_qqq_holdings_series, benchmark_series, time_frame=time_frame_week,mode=mode)
benchmark_minus_top_dia_holdings_week = create_spreads(top_dia_holdings_series, benchmark_series, time_frame=time_frame_week,mode=mode)
benchmark_minus_top_xlk_holdings_week = create_spreads(top_xlk_holdings_series, benchmark_series, time_frame=time_frame_week,mode=mode)
benchmark_minus_top_xlf_holdings_week = create_spreads(top_xlf_holdings_series, benchmark_series, time_frame=time_frame_week,mode=mode)

benchmark_minus_all_series_week= pd.concat([
    benchmark_minus_indices_week,
    benchmark_minus_sectors_week,
    benchmark_minus_industries_week,
    benchmark_minus_top_spy_holdings_week,
    benchmark_minus_top_qqq_holdings_week,
    benchmark_minus_top_dia_holdings_week,
    benchmark_minus_top_xlk_holdings_week,
    benchmark_minus_top_xlf_holdings_week
], axis=1)

benchmark_minus_all_series_week=benchmark_minus_all_series_week.loc[:, ~benchmark_minus_all_series_week.columns.duplicated()]

benchmark_minus_indices_short          = create_spreads(indices_series, benchmark_series, time_frame=time_frame_short,mode=mode)
benchmark_minus_sectors_short          = create_spreads(sectors_series, benchmark_series, time_frame=time_frame_short,mode=mode)
benchmark_minus_industries_short       = create_spreads(industries_series, benchmark_series, time_frame=time_frame_short,mode=mode)
benchmark_minus_top_spy_holdings_short = create_spreads(top_spy_holdings_series, benchmark_series, time_frame=time_frame_short,mode=mode)
benchmark_minus_top_qqq_holdings_short = create_spreads(top_qqq_holdings_series, benchmark_series, time_frame=time_frame_short,mode=mode)
benchmark_minus_top_dia_holdings_short = create_spreads(top_dia_holdings_series, benchmark_series, time_frame=time_frame_short,mode=mode)
benchmark_minus_top_xlk_holdings_short = create_spreads(top_xlk_holdings_series, benchmark_series, time_frame=time_frame_short,mode=mode)
benchmark_minus_top_xlf_holdings_short = create_spreads(top_xlf_holdings_series, benchmark_series, time_frame=time_frame_short,mode=mode)

benchmark_minus_all_series_short= pd.concat([
    benchmark_minus_indices_short,
    benchmark_minus_sectors_short,
    benchmark_minus_industries_short,
    benchmark_minus_top_spy_holdings_short,
    benchmark_minus_top_qqq_holdings_short,
    benchmark_minus_top_dia_holdings_short,
    benchmark_minus_top_xlk_holdings_short,
    benchmark_minus_top_xlf_holdings_short
], axis=1)

benchmark_minus_all_series_short=benchmark_minus_all_series_short.loc[:, ~benchmark_minus_all_series_short.columns.duplicated()]

benchmark_minus_indices_mid          = create_spreads(indices_series, benchmark_series, time_frame=time_frame_mid,mode=mode)
benchmark_minus_sectors_mid          = create_spreads(sectors_series, benchmark_series, time_frame=time_frame_mid,mode=mode)
benchmark_minus_industries_mid        = create_spreads(industries_series, benchmark_series, time_frame=time_frame_mid,mode=mode)
benchmark_minus_top_spy_holdings_mid    = create_spreads(top_spy_holdings_series, benchmark_series, time_frame=time_frame_mid,mode=mode)
benchmark_minus_top_qqq_holdings_mid    = create_spreads(top_qqq_holdings_series, benchmark_series, time_frame=time_frame_mid,mode=mode)
benchmark_minus_top_dia_holdings_mid    = create_spreads(top_dia_holdings_series, benchmark_series, time_frame=time_frame_mid,mode=mode)
benchmark_minus_top_xlk_holdings_mid    = create_spreads(top_xlk_holdings_series, benchmark_series, time_frame=time_frame_mid,mode=mode)
benchmark_minus_top_xlf_holdings_mid    = create_spreads(top_xlf_holdings_series, benchmark_series, time_frame=time_frame_mid,mode=mode)

benchmark_minus_all_series_mid= pd.concat([
    benchmark_minus_indices_mid,
    benchmark_minus_sectors_mid,
    benchmark_minus_industries_mid,
    benchmark_minus_top_spy_holdings_mid,
    benchmark_minus_top_qqq_holdings_short,
    benchmark_minus_top_dia_holdings_mid,
    benchmark_minus_top_xlk_holdings_mid,
    benchmark_minus_top_xlf_holdings_mid
], axis=1)

benchmark_minus_all_series_mid=benchmark_minus_all_series_mid.loc[:, ~benchmark_minus_all_series_mid.columns.duplicated()]


In [9]:
def filter_assets_by_positive_spread_std(asset_spreads):
    spreads = asset_spreads
    positive_spreads = spreads[spreads >= 0] 
    
    mean = positive_spreads.mean()
    std_dev = positive_spreads.std()

    latest_spread = spreads.iloc[-1]
    threshold = mean + std_dev

    return latest_spread>=threshold


def filter_assets_below_negative_std(asset_spreads):
    if not isinstance(asset_spreads, pd.Series):
        raise TypeError("asset_spreads must be a pandas Series")

    negative_spreads = asset_spreads[asset_spreads < 0]
    if negative_spreads.empty:
        return pd.Series(dtype=bool)  
    
    mean_negative = negative_spreads.mean()
    std_dev_negative = negative_spreads.std()

    threshold_negative = mean_negative - 0.75 * std_dev_negative
    return asset_spreads < threshold_negative


In [10]:
filtered_assets = filter_assets_by_positive_spread_std(benchmark_minus_all_series_mid)
filtered_assets = filtered_assets[filtered_assets]
#Create a Plotly Table trace
fig = go.Figure(data=[go.Table(
    header=dict(values=['Date', 'Boolean Value']),
    cells=dict(values=[filtered_assets.index, filtered_assets], align='left'))
])

fig.show()


IndexError: single positional indexer is out-of-bounds

In [ ]:
TICKER = 'PEP'
SPREAD = 'Benchmark_minus_' + TICKER
create_spread_plot(benchmark_minus_all_series_week[SPREAD]).show()
create_spread_plot(benchmark_minus_all_series_short[SPREAD]).show()
create_spread_plot(benchmark_minus_all_series_mid[SPREAD]).show()
plot_risk_adjusted_returns(all_series[TICKER],time_frame_short).show()
plot_risk_adjusted_returns(all_series[TICKER],time_frame_mid).show()